# Projeto 1 - Ciência dos Dados

Nome: Raphael Lahiry de Barros

Nome: Florencia Averame Kramer

### O que temos que fazer?
precisamos criar um dataframe para cada relevância   OK

dentro desses dataframes separar as frases em palavras OK   

analisar a freq relativa de cada palavra --> probabilidade daquela palavra pertencer àquela relevância    

quando quisermos avaliar o teste--> analisamos a probabilidade de cada palavra de uma frase pertencer a uma relevância (usar o smoothing -- aula 8 -- para as palavras que talvez não existam em certas categorias)  

com isso, para cada palavra teremos a probabilidade de que esta pertença a cada relevância  
a relevância com maior probabilidade será a daquela palavra





# Dúvidas
A tabela1 apresenta que os valores são iguais, mas a porcentagem da 2% (já obtivemos 95,7)
A tabela2 apresenta 22,2%

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
## Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com a prova!


### Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,@odonodocabare_ comprei um tablet dia desses e...,0.0
1,@encanadorchapad o negócio é o ps4 slim. peque...,4.0
2,rt @ps3brasil: tohu é anunciado para ps4; deta...,2.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,@leobruto @fridaygus @douglasxavier8 @ocerveje...,0.0


In [4]:
train['Treinamento'] = train['Treinamento'].astype('str')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é o playstation 4. Consideramos relevantes as mensagens que falavam sobre questões técnicas do produto e sobre satisfação, sejam positivas ou negativas. Consideramos irrelevantes as mensagens que só mencionavam o nosso produtos nas '#' ou que eram opiniões pessoais que não agregavam na nossa análise.

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#função que apaga os caracteres especiais e as menções de usuários
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;"\n"()''"",_%$\|/,<>]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    #filtro que remove as menções à usuários com @
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    juncao = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    
    return juncao
    

In [6]:
#aplicando a função cleanup na base treinamento e deixando tudo em letra minúscula
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train['Treinamento'] = train['Treinamento'].str.lower()

In [7]:
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
train['Relevância'].cat.as_ordered()

0      Muito irrelevante
1        Muito relevante
2                 Neutro
3      Muito irrelevante
4      Muito irrelevante
             ...        
746          Irrelevante
747          Irrelevante
748               Neutro
749    Muito irrelevante
750          Irrelevante
Name: Relevância, Length: 751, dtype: category
Categories (5, object): [Muito irrelevante < Irrelevante < Neutro < Relevante < Muito relevante]

### Criando os filtros para criar os dataFrames por relevância

In [8]:
#ligando a relevância à um número
muito_relevante = train['Relevância'] == 'Muito relevante'
relevante = train['Relevância'] == 'Relevante'
neutro = train['Relevância'] == 'Neutro'
irrelevante = train['Relevância'] == 'Irrelevante'
muito_irrelevante = train['Relevância'] == 'Muito irrelevante'

### Criando os dataFrames separados por relevância

In [9]:
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_neutro = train.loc[neutro, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

### Separando cada dataFrame da sua respectiva relevância em listas de palavras

In [10]:
tweets_muito_relevantes = []
lista_vrw = []
for vrw in df_muito_relevante['Treinamento']:
    tweets_muito_relevantes.append(vrw.split(' '))
for vrl in tweets_muito_relevantes:
    lista_vrw += vrl

In [11]:
tweets_relevantes = []
lista_rw = []
for rw in df_relevante['Treinamento']:
    tweets_relevantes.append(rw.split(' '))
for rl in tweets_relevantes:
    lista_rw += rl

In [12]:
tweets_neutros = []
lista_nw = []
for nw in df_neutro['Treinamento']:
    tweets_neutros.append(nw.split(' '))
for nl in tweets_neutros:
    lista_nw += nl

In [13]:
tweets_irrelevantes = []
lista_iw = []
for iw in df_irrelevante['Treinamento']:
    tweets_irrelevantes.append(iw.split(' '))
for il in tweets_irrelevantes:
    lista_iw += il

In [14]:
tweets_muito_irrelevantes = []
lista_viw = []
for viw in df_muito_irrelevante['Treinamento']:
    tweets_muito_irrelevantes.append(viw.split(' '))
for vil in tweets_muito_irrelevantes:
    lista_viw += vil

### Transformando as listas de palavras de cada relevância em pd.Series

In [15]:
serie_muito_relevante = pd.Series(lista_vrw)
serie_relevante = pd.Series(lista_rw)
serie_neutro = pd.Series(lista_nw)
serie_irrelevante = pd.Series(lista_iw)
serie_muito_irrelevante = pd.Series(lista_viw)

### Definindo a quantidade total de palavras na língua portuguesa

In [16]:
#total_train são as palavras existentes nos tweets de treinamento, nos quais verificaremos
#a freq relativa e somaremos um alpha (que representa que a palavra apareceu uma vez) pra garantir
#que as palavras que não existem nos tweets classificados não zerem o classificador
total_train = lista_vrw + lista_rw + lista_nw + lista_iw + lista_viw

#na fórmula precisamos dividir pela quantidade de palavras existentes na lingua portuguesa
#para obter esse valor somamos a quantidade de palavras existentes nos tweets classificados 
#com um "chute" das palavras existentes que não estão em nenhum tweet classificado
alpha = 1
v = 1e4
lingua_portuguesa = len(total_train) + alpha*v

'''
port = pd.Series(total_train)
prob_port = port.value_counts(True) + (alpha/lingua_portuguesa)'''

'\nport = pd.Series(total_train)\nprob_port = port.value_counts(True) + (alpha/lingua_portuguesa)'

### Pd.series das palavras que aparecem nos tweets classificados

In [17]:
'''serie_total_train = pd.Series(total_train)
#serie_total_train.value_counts()'''

'serie_total_train = pd.Series(total_train)\n#serie_total_train.value_counts()'

### Frequências relativas e absolutas das palavras de cada relevância

In [18]:
#muito relevante
freq_ab_mr = serie_muito_relevante.value_counts() 
freq_re_mr = (freq_ab_mr + alpha)/(freq_ab_mr.sum() + (alpha*v)) #serie_muito_relevante.value_counts(True) 

In [19]:
#relevante
freq_ab_r = serie_relevante.value_counts() 
freq_re_r = (freq_ab_r + alpha)/(freq_ab_r.sum() + (alpha*v)) #serie_relevante.value_counts(True) 

In [20]:
#neutro
freq_ab_n = serie_neutro.value_counts() 
freq_re_n = (freq_ab_n + alpha)/(freq_ab_n.sum() + (alpha*v)) #serie_neutro.value_counts(True)

In [21]:
#irrelevante
freq_ab_i = serie_irrelevante.value_counts() 
freq_re_i = (freq_ab_i + alpha)/(freq_ab_i.sum() + (alpha*v)) #serie_irrelevante.value_counts(True) 

In [22]:
#muito irrelevante
freq_ab_mi = serie_muito_irrelevante.value_counts() 
freq_re_mi = (freq_ab_mi + alpha)/(freq_ab_mi.sum() + (alpha*v)) #serie_muito_irrelevante.value_counts(True) 

## Montando Naive-Bayes

### Probabilidade de cada relevância à priori

In [23]:
#contagem de tweets
p_mr = len(tweets_muito_relevantes) / len(train["Treinamento"])
p_r = len(tweets_relevantes) / len(train["Treinamento"])
p_n = len(tweets_neutros) / len(train["Treinamento"])
p_i = len(tweets_irrelevantes) / len(train["Treinamento"])
p_mi = len(tweets_muito_irrelevantes) / len(train["Treinamento"])

print(p_mr + p_r + p_n + p_i + p_mi)

0.9986684420772303


### Função que retorna a probabilidade de uma frase pertencer à uma cateogria específica de relevância com base nas palavras que a compõem

In [24]:
def datawitter(frase):
    
    prob_frase_dado_mr = 1
    prob_frase_dado_r = 1
    prob_frase_dado_n = 1
    prob_frase_dado_i = 1
    prob_frase_dado_mi = 1
    
    frase = frase.split(' ')
    
    for palavra in frase:
        if palavra in freq_re_mr:
            prob_frase_dado_mr *= freq_re_mr[palavra]
        elif palavra not in freq_re_mr:
            prob_frase_dado_mr *= (alpha/(freq_ab_mr.sum() + (alpha*v))) 
            
        if palavra in freq_re_r:
            prob_frase_dado_r *= freq_re_r[palavra]
        elif palavra not in freq_re_r:
            prob_frase_dado_r *= (alpha/(freq_ab_r.sum() + (alpha*v)))      
            
        if palavra in freq_re_n:
            prob_frase_dado_n *= freq_re_n[palavra]
        elif palavra not in freq_re_n:
            prob_frase_dado_n *= (alpha/(freq_ab_n.sum() + (alpha*v)))
            
        if palavra in freq_re_i:
            prob_frase_dado_i *= freq_re_i[palavra]
        elif palavra not in freq_re_i:
            prob_frase_dado_i *= (alpha/(freq_ab_i.sum() + (alpha*v)))
            
        if palavra in freq_re_mi:
            prob_frase_dado_mi *= freq_re_mi[palavra]
        elif palavra not in freq_re_mi:
            prob_frase_dado_mi *= (alpha/(freq_ab_mi.sum() + (alpha*v)))
            
      #soma de contagens naquela cat  
    prob_mr_dado_frase = prob_frase_dado_mr*p_mr
    #print(prob_mr_dado_frase)
    prob_r_dado_frase = prob_frase_dado_r*p_r
    #print(prob_r_dado_frase)
    prob_n_dado_frase = prob_frase_dado_n*p_n
    #print(prob_n_dado_frase)
    prob_i_dado_frase = prob_frase_dado_i*p_i
    #print(prob_i_dado_frase)
    prob_mi_dado_frase = prob_frase_dado_mi*p_mi
    #print(prob_mi_dado_frase)
    
    
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_mr_dado_frase, 10):
        return 'Muito relevante'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_r_dado_frase, 10): 
        return 'Relevante'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_n_dado_frase, 10): 
        return 'Neutro'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_i_dado_frase, 10):
        return 'Irrelevante'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_mi_dado_frase, 10): 
        return 'Muito irrelevante'
    
    '''
    if max(prob_mr_dado_frase, prob_r_dado_frase, prob_n_dado_frase, prob_i_dado_frase, prob_mi_dado_frase) == prob_mr_dado_frase:
        return 'Muito relevante'
    if max(prob_mr_dado_frase, prob_r_dado_frase, prob_n_dado_frase, prob_i_dado_frase, prob_mi_dado_frase) == prob_r_dado_frase: 
        return 'Relevante'
    if max(prob_mr_dado_frase, prob_r_dado_frase, prob_n_dado_frase, prob_i_dado_frase, prob_mi_dado_frase) == prob_n_dado_frase: 
        return 'Neutro'
    if max(prob_mr_dado_frase, prob_r_dado_frase, prob_n_dado_frase, prob_i_dado_frase, prob_mi_dado_frase) == prob_i_dado_frase:
        return 'Irrelevante'
    if max(prob_mr_dado_frase, prob_r_dado_frase, prob_n_dado_frase, prob_i_dado_frase, prob_mi_dado_frase) == prob_mi_dado_frase: 
        return 'Muito irrelevante'
    '''

In [26]:
datawitter('o negócio é o ps4 slim pequeno e silencioso econômico e pra transportar é igual a um notebook')

'Irrelevante'

In [27]:
train['Treinamento'][1]

'o negócio é o ps4 slim pequeno e silencioso econômico e pra transportar é igual a um notebook'

### Coluna do dataFrame que contem a nossa classificação

In [28]:
#del train['Classificacao']

In [29]:
train['Classificacao'] = ''

### Preenchimento da coluna contendo a nossa classificação

In [30]:
for frase in train['Treinamento']:
    train['Classificacao'][(train[train['Treinamento']==frase].index.values)] = datawitter(frase)

<ipython-input-30-4cdc29e04c33>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Classificacao'][(train[train['Treinamento']==frase].index.values)] = datawitter(frase)


In [31]:
train["Classificacao"]

0            Irrelevante
1            Irrelevante
2                 Neutro
3      Muito irrelevante
4      Muito irrelevante
             ...        
746          Irrelevante
747          Irrelevante
748               Neutro
749    Muito irrelevante
750          Irrelevante
Name: Classificacao, Length: 751, dtype: object

In [32]:
#train['Classificacao'] = train['Classificacao'].astype('category')
#train['Classificacao'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
#train['Classificacao'].cat.as_ordered()

In [33]:
train['Classificacao'].value_counts(sort=False)

Muito irrelevante    166
Muito relevante        4
Neutro               166
Relevante            108
Irrelevante          307
Name: Classificacao, dtype: int64

### Tabela que retorna a eficácia do nosso classificador

In [34]:
tabela = pd.crosstab(train['Relevância'], train['Classificacao'], normalize='all')
tabela

Classificacao,Irrelevante,Muito irrelevante,Muito relevante,Neutro,Relevante
Relevância,,,,,
Muito irrelevante,0.053333,0.214667,0.000000,0.009333,0.001333
Irrelevante,0.290667,0.004000,0.000000,0.006667,0.000000
Neutro,0.017333,0.001333,0.000000,0.192000,0.000000
Relevante,0.017333,0.001333,0.000000,0.004000,0.137333
Muito relevante,0.029333,0.000000,0.005333,0.009333,0.005333


### Eficácia do nosso classificador

In [35]:
#quanto o classificador acertou?
f'{(tabela.iloc[0,1]+tabela.iloc[1,0]+tabela.iloc[2,3]+tabela.iloc[3,4]+tabela.iloc[4,2])*100:.5g}%'

'84%'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

### Transformando o dataFrame teste em string para poder aplicar a função cleanup e deixar o texto todo em minúscula

In [36]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Teste'] = test['Teste'].astype('str')
test['Teste'] = test['Teste'].apply(cleanup)
test['Teste'] = test['Teste'].str.lower()

### Transformando a coluna 'Relevância' em categórica

In [42]:
test['Relevância'] = test['Relevância'].astype('category')
test['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
test['Relevância'].cat.as_ordered()

0            Irrelevante
1            Irrelevante
2            Irrelevante
3            Irrelevante
4      Muito irrelevante
             ...        
496    Muito irrelevante
497          Irrelevante
498    Muito irrelevante
499    Muito irrelevante
500            Relevante
Name: Relevância, Length: 501, dtype: category
Categories (5, object): [Muito irrelevante < Irrelevante < Neutro < Relevante < Muito relevante]

### Preenchendo a coluna que contêm as classificações do nosso classificador

In [43]:
test['Classificador'] = ''
for frase in test['Teste']:
    test['Classificador'][(test[test['Teste']==frase].index.values)] = datawitter(frase)


<ipython-input-43-e23d722351d7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Classificador'][(test[test['Teste']==frase].index.values)] = datawitter(frase)


In [44]:
#test['Classificador'].cat.as_ordered()

In [45]:
test

,Teste,Relevância,Classificador
0,simmm sou apaixonada em jogos assim tipo unich...,Irrelevante,Irrelevante
1,atleta nato podemos ver o biceps que ergue o c...,Irrelevante,Irrelevante
2,kkkk vc joga onde no xbox ps4 ou pc,Irrelevante,Muito irrelevante
3,eu sou ps4,Irrelevante,Irrelevante
4,path of exile ps4 farmando e fazendo o atlas 4...,Muito irrelevante,Relevante
...,...,...,...
496,meus progressos em troféus nos jogos que tenho...,Muito irrelevante,Irrelevante
497,nada que uma decepção amorosa faça que meu ps4...,Irrelevante,Irrelevante
498,alguém me empresta um ps4 aí por umas 3hrs,Muito irrelevante,Irrelevante
499,“trampo foi corrido” “ ontem o levi começou a ...,Muito irrelevante,Irrelevante


In [46]:
tabela2 = pd.crosstab(test['Relevância'], test['Classificador'], normalize='all')
tabela2

Classificador,Irrelevante,Muito irrelevante,Neutro,Relevante
Relevância,,,,
Muito irrelevante,0.140,0.032,0.030,0.014
Irrelevante,0.292,0.028,0.034,0.030
Neutro,0.114,0.002,0.088,0.016
Relevante,0.070,0.000,0.064,0.018
Muito relevante,0.016,0.000,0.006,0.006


In [47]:
#quanto o classificador acertou?

f'{(tabela2.iloc[0,1]+tabela2.iloc[1,0]+tabela2.iloc[2,2]+tabela2.iloc[3,3]+0)*100:.5g}%'

'43%'

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**